In [12]:
import pandas as pd
import copy

#Need_data_store


In [13]:
dt ={
    "AND":['0','8'], 
    "ADD":['1','9'],
    "LDA":['2','A'],
    "STA":['3','B'],
    "BUN":['4','C'],
    "BSA":['5','D'],
    "ISZ":['6','E'],
    "CLA":['7800'],
    "CLE":['7400'],
    "CMA":['7200'],
    "CME":['7100'],
    "CIR":['7080'],
    "CIL":['7040'],
    "INC":['7020'],
    "SPA":['7010'],
    "SNA":['7008'],
    "SZA":['7004'],
    "SZE":['7002'],
    "HLT":['7001'],
    "INP":['F800'],
    "OUT":['F400'],
    "SKI":['F200'],
    "SKO":['F100'],
    "INO":['F080'],
    "IOF":['F040'],
     "ORG":['[0000..00FF]']
     }
data_type = ['HEX', 'DEC', 'OCT', 'BIN']


#MAKE

In [105]:
class Trans_Asem:
  def __init__(self): #init MAX
    self.dt = dt
    self.MEMORY_MEX_SIZE = 65536 #16 WIDTH
    self.memory_position = 256   #100
    self.memory_tran = {}
    self.decoded_tran = {}
    self.var = {}
    self.oper = {}
    self.cache = {}
    return 

  def get_user_input(self, file_address):
    f = open(file_address, 'r') #file open
    while True:
      line = f.readline()
      if not line: break
      if len(line.split()) == 0:continue
      if line.split()[0] == "ORG":             #ORG : 0000;
        self.decoded_tran[self.dt["ORG"][0]] = "0000"
        continue
      #var
      if ',' in line: # 변수체크
        temp = line.upper().replace(',','').split()
        key = str(hex(self.memory_position)).replace('0x','').zfill(4) #0x0000 fill
        self.var[key] = temp
        self.memory_tran[key] = temp  # 
        self.memory_position += 1     #100 101 102 103

      else:
        temp = line.upper().split()
        key = str(hex(self.memory_position)).replace('0x','').zfill(4)
        self.oper[key] = temp
        self.memory_tran[key] = temp
        self.memory_position += 1
    f.close()
    return 

  def decode(self): #find_var and decode dictionary
    cp = copy.deepcopy(self.var) #temp[var] open(deepcopy)   
    while len(cp) != 0: #len(cp)의 크기가 0이 될때까지 decode 시작
      print(cp)
      for key in cp.keys(): #var key 순서대로 불러오기
        value = cp[key]
        if len(value) == 3: #BUN SUB I <<- i 로 메모리 이름 바뀌므로 탐색
          temp = value[2]
          if value[1] == 'HEX': #16진수 처리
            temp = temp.replace('0x','').zfill(4)
            self.cache[value[0]] = [temp, key]
            del cp[key]
            break
          elif value[1] == 'DEC': #10진수 처리
            if int(temp) < 0:
              temp = self.MEMORY_MEX_SIZE + int(temp) # 65536 - 5 = FFFB
            temp = hex(temp) #10진수 -> 16진수로 변환
            temp = temp.replace('0x','').zfill(4)
            self.cache[value[0]] = [temp,key]
            del cp[key]
            break
          elif value[1] == 'OCT': #8진수 처리
            temp = hex(int(temp, 8))
            temp = temp.replace('0x','').zfill(4)
            self.cache[value[0]] = [temp,key]
            del cp[key]
            break
          elif value[1] == 'BIN': #2진수 처리
            temp = hex(int(temp, 2))
            temp = temp.replace('0x','').zfill(4)
            self.cache[value[0]] = [temp, key]
            del cp[key]
            break 
          elif value[1] in self.dt: #dt 기본값들을 탐색
            if temp in self.cache: 
              if self.cache[temp][1][0] == "0":
                temp = self.dt[value[1]][0] + self.cache[temp][1][1:]
              else:
                temp = self.dt[value[1]][0] + self.cache[temp][1]
              self.cache[value[0]] = [temp, key]
              del cp[key]
              break
        elif len(value) == 2:
          if value[1] in self.cache:
            self.cache[value[0]] = [self.cache[value[1]][1], key]
            del cp[key]
            break
          elif value[1] in self.dt:
            self.cache[value[0]] = [self.dt[value[1]][0], key]
            del cp[key]
            break
        else:
          self.cache[value[0]] = ["0000",key]
          del cp[key]
          break
    return

  def translate(self):
    for key in self.memory_tran.keys():
      temp = ""
      values = self.memory_tran[key]
      flag = False # flag
      if values[0] in self.cache:
        self.decoded_tran[key] = self.cache[values[0]][0].upper() 
        continue
      else:
        for v in reversed(values): # 뒤부터 탐색
          if v == 'I':
            flag = True ## 맨 뒤 i 가 붙을 경우 setup이 달라짐
            continue 
          if v in self.dt:
            if flag: #True
              temp = self.dt[v][1] + temp #ex: LDA {2} or {A} <-this
            else:
              temp = self.dt[v][0] + temp #ex: LDA {2}<-this or {A}  
          elif v in self.cache: #cache (변수 저장 공간) 에 있을경우
            t = self.cache[v][1] # {}
            if self.cache[v][1][0] == "0": #0100
              temp = self.cache[v][1][1:] + temp 
          self.decoded_tran[key] = temp.upper() #temp로 대체
      if self.decoded_tran[key] == "":
         self.decoded_tran[key] = "0000"

  def result(self):
    fp = open("result.txt.mif",'w')
    fp.write("WIDTH = 16; \n")
    fp.write("DEPTH=4096; \n\n")
    fp.write("ADDRESS_RADIX=HEX; \n")
    fp.write("DATA_RADIX=HEX; \n\n")
    fp.write("CONTENT BEGIN;\n")
    for key in self.decoded_tran.keys():
      data = '\t%s:\t%s; \n' %(key, self.decoded_tran[key])
      fp.write(data)
    fp.write("END;")
    fp.close()

  def check(self):
    print("WIDTH=16;")
    print("DEPTH=4096;")
    print()
    print("ADDRESS_RADIX=HEX;")
    print("DATA_RADIX=HEX;")
    print()
    print("CONTENT BEGIN;")
    for key in self.decoded_tran.keys():
      print('\t%s:\t%s;'%(key, self.decoded_tran[key]))
    print("END;")
    


#RESULT


In [106]:
t = Trans_Asem()
t.get_user_input("/content/drive/MyDrive/Sys_Asem/DigCom-A1.2-Assembler/res_test/test_case.txt")

In [107]:
t.decode()

{'0100': ['CIF', 'SKI'], '0103': ['COF', 'SKO'], '0108': ['CHR']}
{'0103': ['COF', 'SKO'], '0108': ['CHR']}
{'0108': ['CHR']}


In [108]:
t.cache

{'CIF': ['F200', '0100'], 'COF': ['F100', '0103'], 'CHR': ['0000', '0108']}

In [109]:
t.memory_tran

{'0100': ['CIF', 'SKI'],
 '0101': ['BUN', 'CIF'],
 '0102': ['INP'],
 '0103': ['COF', 'SKO'],
 '0104': ['BUN', 'COF'],
 '0105': ['OUT'],
 '0106': ['STA', 'CHR'],
 '0107': ['HLT'],
 '0108': ['CHR'],
 '0109': ['END']}

In [112]:
t.translate()

In [113]:
t.decoded_tran

{'[0000..00FF]': '0000',
 '0100': 'F200',
 '0101': '4100',
 '0102': 'F800',
 '0103': 'F100',
 '0104': '4103',
 '0105': 'F400',
 '0106': '3108',
 '0107': '7001',
 '0108': '0000',
 '0109': '0000'}

In [114]:
t.check()

WIDTH=16;
DEPTH=4096;

ADDRESS_RADIX=HEX;
DATA_RADIX=HEX;

CONTENT BEGIN;
	[0000..00FF]:	0000;
	0100:	F200;
	0101:	4100;
	0102:	F800;
	0103:	F100;
	0104:	4103;
	0105:	F400;
	0106:	3108;
	0107:	7001;
	0108:	0000;
	0109:	0000;
END;


In [115]:
t.result()